In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from twpasolver import TWPAnalysis
from twpasolver.mathutils import dBm_to_I
from twpasolver.models import TWPA

plt.rcParams["font.size"] = 13.5
plt.rcParams["axes.axisbelow"] = True

In [ ]:
twpa_file = "model_cpw_dartwars_13nm_Lk8_5.json"
twpa = TWPA.from_file(twpa_file)
a = TWPAnalysis(twpa=twpa, freqs_arange=(0.3, 10, 0.5e-3), data_file="data")
ax = a.plot_response()

In [ ]:
_ = a.phase_matching()
a.plot_phase_matching(thin=3)

In [ ]:
s_arange = np.arange(1, 7, 0.05)
_ = a.gain(signal_freqs=s_arange, Is0=1e-6)
a.plot_gain()

In [ ]:
_ = a.bandwidth()
plt.plot(a.data["gain"]["signal_freqs"], a.data["gain"]["gain_db"])
plt.axhline(a.data["bandwidth"]["reduced_gain"], c="black", ls="--")
for edg in a.data["bandwidth"]["bandwidth_edges"]:
    plt.axvline(edg, color="black", ls="--")
plt.axhline(a.data["bandwidth"]["mean_gain"], c="black", ls="--");

In [ ]:
optimal = a.data["optimal_pump_freq"]
pumps = np.arange(optimal - 0.2, optimal + 0.5, 0.02)
pumpsweep_res = a.parameter_sweep(
    "gain", "pump", pumps, signal_freqs=s_arange, save_name="pump_profile"
)

In [ ]:
plt.pcolor(pumpsweep_res["signal_freqs"][0], pumps, pumpsweep_res["gain_db"])
plt.xlabel("Signal frequency [GHz]")
plt.ylabel("Pump frequency [GHz]")
c = plt.colorbar()
c.set_label("Gain [dB]")

In [ ]:
signals_db = np.arange(-80, -40, 1)
signals = dBm_to_I(signals_db)
edges = a.data["bandwidth"]["bandwidth_edges"]
s_arange = a.data["bandwidth"]["bw_freqs"]  # (edges[0], edges[-1], 0.1)
compression_res = a.parameter_sweep(
    "gain", "Is0", signals, signal_freqs=s_arange, thin=1000, save_name="compression"
)

In [ ]:
mean_gains = np.mean(compression_res["gain_db"], axis=1)
reduced_1db = mean_gains[0] - 1
cp_1db = np.interp(reduced_1db, mean_gains[::-1], signals_db[::-1])
plt.scatter(signals_db, mean_gains)
plt.xlabel("Signal power [dB]")
plt.ylabel("Gain [dB]")
plt.axhline(reduced_1db, c="black", ls="--")
plt.axvline(cp_1db, c="black", ls="--")

In [ ]:
a.save_data()